In [ ]:
!pip -q install transformers peft accelerate bitsandbytes datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip -q install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    TaskType
)
import pandas as pd
from trl import SFTTrainer


In [ ]:
model_id = "vilm/vinallama-7b-chat"
device = torch.device("cuda")
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.80G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

In [ ]:

# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
# Load and prepare the dataset
def load_dataset(file_path):
    df = pd.read_excel(file_path)

    # Format the data into instruction format
    def format_instruction(row):
        return f"### Question: {row['Question']}\n\n### Answer: {row['Answering']}"

    df['text'] = df.apply(format_instruction, axis=1)
    return Dataset.from_pandas(df[['text']])

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors=None
    )
# Load and preprocess the dataset
dataset = load_dataset("/content/drive/MyDrive/Data/tdtu_data.xlsx")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['text'][0]

'### Question: Có mấy nội dung thang điểm đánh giá rèn luyện tại tdtu\n\n### Answer: Có 6 nội dung và thang điểm đánh giá điểm rèn luyện tại trường Đại học Tôn Đức Thắng: 1. Việc tuân thủ kỷ luật, nề nếp (tối đa 20 điểm), 2. Việc tham gia các hoạt động hoạt động chuyên môn học thuật (tối đa 20 điểm), 3. Việc tham gia các hoạt động văn hóa, nghệ thuật, văn nghệ, thể dục thể thao, hoạt động phong trào (tối đa 20 điểm), 4. Việc tham gia các hoạt động cộng đồng, công tác xã hội, công tác tình nguyện (tối đa 20 điểm), 5. Tinh thần tiên phong, gương mẫu (tối đa 20 điểm), 6. Điểm thưởng (tối đa 20 điểm)'

In [ ]:
# Define a proper data collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We're not using masked language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./vina-llama-7b-chat-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    save_total_limit=3,
    push_to_hub=False,
    warmup_ratio=0.03
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator

)
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=6, training_loss=2.527010917663574, metrics={'train_runtime': 147.2552, 'train_samples_per_second': 0.611, 'train_steps_per_second': 0.041, 'total_flos': 1845317515345920.0, 'train_loss': 2.527010917663574, 'epoch': 3.0})